In [1]:
pip install openpyxl, pandasql

ERROR: Invalid requirement: 'openpyxl,'
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
from pandasql import sqldf

# Primeiro, carreguei as tabelas do Exercicio 4 e 5 do xlsx que estava na pasta.  
## Fiz isso para poder executar o sql dentro do jupyter notebook

In [3]:
customer_raw = pd.read_excel("challenge_tables.xlsx",sheet_name="customer")
account_raw = pd.read_excel("challenge_tables.xlsx",sheet_name="account")
bankslip_raw = pd.read_excel("challenge_tables.xlsx",sheet_name="bankslip")
pix_send_raw = pd.read_excel("challenge_tables.xlsx",sheet_name="pix_send")
pix_received_raw = pd.read_excel("challenge_tables.xlsx",sheet_name="pix_received")
p2p_tef_raw = pd.read_excel("challenge_tables.xlsx",sheet_name="p2p_tef")

Depois de carregados, usei a primeira coluna de cada para serem as novas colunas de cada tabela

In [4]:
customer = pd.DataFrame(columns=customer_raw["column name"].values.tolist())
account = pd.DataFrame(columns=account_raw["column name"].values.tolist())
bankslip = pd.DataFrame(columns=bankslip_raw["column name"].values.tolist())
pix_send = pd.DataFrame(columns=pix_send_raw["column name"].values.tolist())
pix_received = pd.DataFrame(columns=pix_received_raw["column name"].values.tolist())
p2p_tef = pd.DataFrame(columns=p2p_tef_raw["column name"].values.tolist())

In [5]:
customer

,customer_id,name,document_number,created_at,updated_at,phone_number,address,timestamp,event_id,dt


In [6]:
account

,account_id,customer_id,created_at,updated_at,status,timestamp,event_id,dt


In [7]:
bankslip

,amount,account_id,beneficiary_name,barcode,due_date,timestamp,event_id,dt


In [8]:
pix_send

,amount,account_id,type_document_number_source_holder,document_number_source_holder,type_document_number_destination_holder,document_number_dentination_holder,user_message,destination_key,timestamp,event_id,dt


In [9]:
pix_received

,amount,account_id,type_document_number_source_holder,document_number_source_holder,type_document_number_destination_holder,document_number_dentination_holder,user_message,timestamp,event_id,dt


In [10]:
p2p_tef

,amount,account_id_source,account_id_destination,timestamp,event_id,dt


# Exercicio 4
Como havia não tinha dados para ir desenvolvendo e testando, acreditei que o que estava fazendo não ia dar certo,  
Portanto, o resultado mais proximo que cheguei foi assim:  
Em vez de uma so coluna de transaction_type e mean_value, adicionei uma coluna para cada transaction_type e uma com o AVG(amount) desse tipo.  
A tabela final ficaria com multiplas colunas de transaction_type e mean_value em vez de uma linha para cada, fornecendo assim, potencialmente o mesmo resultado final

In [11]:
query = """
SELECT c.customer_id, a.account_id, c.name, c.dt as date, 
'bankslip' AS transaction_type_b, AVG(b.amount) AS mean_value_b,
'pix_send' AS transaction_type_p, AVG(p.amount) AS mean_value_p,
'p2p_tef' AS transaction_type_p2p_tef, AVG(t.amount) AS mean_value_p2p_tef
FROM customer c 
INNER JOIN account a ON c.customer_id = a.account_id AND c.dt = a.dt
INNER JOIN bankslip b ON b.account_id = a.account_id AND b.dt = a.dt 
INNER JOIN pix_send p ON b.account_id = a.account_id AND p.dt = a.dt
INNER JOIN p2p_tef t ON t.account_id_source = a.account_id AND t.dt = a.dt 
GROUP BY c.name
"""
sqldf(query, globals())

,customer_id,account_id,name,date,transaction_type_b,mean_value_b,transaction_type_p,mean_value_p,transaction_type_p2p_tef,mean_value_p2p_tef


# Exercicio 5
No exercicio 5 pensei no desenvolvimento o como é feito atualmente na empresa que estou,  
Criando obt (one big table) com todos os campos que podem ser necessarios, particionando eles por data,  
Assim o usuário deve pesquisar nessa tabela somente pela chave na particao de data que ele busca.  
Não tenho os número de benchmark que mostram que essa solução é superior ou inferior as outras, decidi usar ela como solução pois é a que atualmente ja uso 

In [12]:
# Inicialmente, coleta-se todos os campos, adicionando um sulfixo em cada um para saber de onde ele veio na obt
result = [] 
for c in customer.columns:    
    result.append("c."+c+", ")    
for c in account.columns:
    result.append("a."+c+" AS "+c+"_account, ")
for c in bankslip.columns:
    result.append("b."+c+" AS "+c+"_bankslip, ")
for c in pix_send.columns:
    result.append("p."+c+" AS "+c+"_pix_send, ")       
for c in pix_received.columns:
    result.append("pr."+c+" AS "+c+"_pix_received, ")   
for c in p2p_tef.columns:
    result.append("t."+c+" AS "+c+"_p2p_tef, ")
i = " "
for item in result:
    i = i + item
i = i[:-2]

In [13]:
# depois executa a query unindo todos os dados
query = """
SELECT """+i+"""
FROM customer c 
INNER JOIN account a ON c.customer_id = a.account_id AND c.dt = a.dt
INNER JOIN bankslip b ON b.account_id = a.account_id AND b.dt = a.dt 
INNER JOIN pix_send p ON b.account_id = a.account_id AND p.dt = a.dt
INNER JOIN pix_received pr ON b.account_id = a.account_id AND p.dt = a.dt
INNER JOIN p2p_tef t ON t.account_id_source = a.account_id AND t.dt = a.dt 
"""
obt = sqldf(query, globals())

In [14]:
obt.info()

<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Data columns (total 53 columns):
 #   Column                                                Non-Null Count  Dtype 
---  ------                                                --------------  ----- 
 0   customer_id                                           0 non-null      object
 1   name                                                  0 non-null      object
 2   document_number                                       0 non-null      object
 3   created_at                                            0 non-null      object
 4   updated_at                                            0 non-null      object
 5   phone_number                                          0 non-null      object
 6   address                                               0 non-null      object
 7   timestamp                                             0 non-null      object
 8   event_id                                              0 non-null      object
 9   dt   